# Week 10 Quiz

## Getong Zhong - gz2338

### Instructions

Replace the Name and UNI in cell above and the notebook name

Replace all '____' below using the instructions provided.

When completed, 
 - make sure you've replaced Name and UNI in the first cell and filename (eg: quiz_09-hw2592)
 - Click the github link shared on canvas for quiz10 to generate your own repository of quiz10
 - Click your own repository of quiz10 to commit your answers to github

In this quiz, we're going to load documents from 2 categories (space, cars) in the 20newsgroups dataset.

The goal is to train a classifier that classifies documents into these 2 categories based on a term frequency representation of the documents.

We will then calculate mean cross-validaion accuracy of a RandomForestClassifier using this transformation.

### Setup Environment

In [1]:
import numpy as np
import pandas as pd

### Load the Dataset

In [5]:
# Import fetch_20newsgroups from sklearn.datasets
from sklearn.datasets import fetch_20newsgroups

# Load the dataset using fetch_20newsgroups().
#   Only fetch the two categories of interest using categories=['sci.space','rec.autos']
# Store in the result into newsgroups
newsgroups = fetch_20newsgroups(subset='all', categories=['sci.space', 'rec.autos'], shuffle=True, random_state=42)

# Store the newsgroups.data as docs, newsgroups.target as y and newsgroups.target_names as y_names
docs = newsgroups.data
y = newsgroups.target
y_names = newsgroups.target_names

# Print the number of observations by printing the length of docs
print(len(docs))

1977


In [6]:
# Print the text of the first document in docs.
# Note: try printing both with and without the print() statement
#  with the print statement, linebreaks are parsed,
#  without, linebreaks are printed as excape characters
print(docs[0])
docs[0]

From: hathaway@stsci.edu
Subject: Re: Vandalizing the sky.
Lines: 32
Organization: Space Telescope Science Institute
Distribution: na

In article <C65LJ5.5Az@fs7.ece.cmu.edu>, loss@fs7.ECE.CMU.EDU (Doug Loss) writes:
>    I didn't want to quote all the stuff that's been said recently, I
> just wanted to add a point.
> 
.. 
> then enforces those rights for them.  Here in the U.S., the constitution
> provides a "Bill of Rights" from which most if not all legal rights are
> considered to derive.  I'm sure that most other countries have

These seem hardly like the groups to discuss this in, but HUH??? 
All legitimate power to enforce these rights derives from the consent 
of the governed, not from no steenkin' piece of paper.  Civilized gov'mnt 
is not an autonomous computer program, it's interactive.  The Constitution 
was made by the people and can be trashed by us - it ain't no sacred 
scripture from which rights flow.  Our 'rights' come from our souls. 
And I sure didn't see any reques

'From: hathaway@stsci.edu\nSubject: Re: Vandalizing the sky.\nLines: 32\nOrganization: Space Telescope Science Institute\nDistribution: na\n\nIn article <C65LJ5.5Az@fs7.ece.cmu.edu>, loss@fs7.ECE.CMU.EDU (Doug Loss) writes:\n>    I didn\'t want to quote all the stuff that\'s been said recently, I\n> just wanted to add a point.\n> \n.. \n> then enforces those rights for them.  Here in the U.S., the constitution\n> provides a "Bill of Rights" from which most if not all legal rights are\n> considered to derive.  I\'m sure that most other countries have\n\nThese seem hardly like the groups to discuss this in, but HUH??? \nAll legitimate power to enforce these rights derives from the consent \nof the governed, not from no steenkin\' piece of paper.  Civilized gov\'mnt \nis not an autonomous computer program, it\'s interactive.  The Constitution \nwas made by the people and can be trashed by us - it ain\'t no sacred \nscripture from which rights flow.  Our \'rights\' come from our souls. \nA

In [7]:
# Print the target value of the first document in y.
print(y[0])

1


In [8]:
# Print the target_name of the first document using y_names and y.
print(y_names[y[0]])

sci.space


### Use CountVectorizer to Convert To TF

In [11]:
# Import CountVectorizer from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Initialize a CountVectorizer object. It should
#   lowercase all text, 
#   include both unigrams and bigrams
#   exclude terms that occur in fewer than 10 documents
#   exclude terms that occur in more than 95% of documents
# Store as cvect
cvect = CountVectorizer(lowercase=True, ngram_range=(1, 2), min_df=10, max_df=0.95)

# Fit cvect on docs and transform docs into their term frequency representation.
# Store as X_tf
X_tf = cvect.fit_transform(docs)

# Print the shape of X_tf. 
# The number of rows should match the number of documents 
#    and the number of columns should be in the thousands
print(X_tf.shape)

(1977, 9483)


In [12]:
# Print out the last 5 terms in the learned vocabulary in cvect
#   using .get_feature_names() which returns a list of terms corresponding
#   to the order of the columns in X_tf
# They should all be related to zoos
feature_names = cvect.get_feature_names()
print([term for term in feature_names if "zoo" in term][-5:])

['zoo toronto', 'zoology', 'zoology between', 'zoology kipling', 'zoology lines']


C:\Users\tonyg\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# The stopwords learned by cvect are stored as a set in cvect.stop_words_
# We'd like to print out a small subset of these terms.
# One way to get a subset of a set is to treat it as a list.
# First, convert the stop_words_ set to a list.
# Store as stop_words_list
stop_words_list = list(cvect.stop_words_)

# Print out the first 5 elements in stop_words_list.
# Note that, since a set is unordered, 
#    there is no meaning to the ordering of these terms and they may vary over runs.
print(stop_words_list[:5])

['verne collector', 'team and', 'budget enviroment', 'gordon2 on', 'the gobs']


### Calculate Mean CV Accuracy Using RandomForestClassifier

In [14]:
# Import cross_val_score from sklearn
from sklearn.model_selection import cross_val_score


# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Get a set of 5-fold CV scores using
#    a RandomForestClassifier with 50 trees and n_jobs=-1
#    and the full dataset X_tf and y
# Store as cv_scores
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)
cv_scores = cross_val_score(rf, X_tf, y, cv=5)
# Print the mean of these cv_scores. The mean accuracy should be above .9
print(cv_scores.mean())

0.9742079018028384


### Optional: Find Important Features

In [ ]:
# CountVectorizer stores the feature names (terms in the vocabulary) in two ways:
#  1. as a dictionary of term:colum_index pairs, accessed via cvect.vocabulary_
#  2. as a list of terms, in column index order, accessed via cvect.get_feature_names()
#
# We can get the indices of the most important features by retraining a RandomForestClassifier on X_tf,y
#  and accessing .feature_importances_
#
# Using some combination of the above data-structures, 
#  print out the top 10 terms in the vocabulary
#  ranked by the feature importances learned by a RandomForestClassifier with 50 trees
# 
# The terms you find will likely not be surprising given the document categories.
____